In [ ]:
#!pip install river

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ------------------- -------------------- 1.0/2.2 MB 64.6 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.2 MB 64.6 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.2 MB 64.6 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.2 MB 64.6 MB/s eta 0:00:01
   ---------------------------------------  2.2/2.2 MB 9.8 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 8.6 MB/s eta 0:00:00


In [ ]:
#from river import stream, datasets

from river.naive_bayes import GaussianNB
from river.preprocessing import StandardScaler
from river.drift.binary import DDM
from river.metrics import Accuracy
from river.datasets import synth

In [ ]:
# 1. 데이터 스트림 로딩 (Agrawal with concept drift)
dataset = synth.Agrawal(classification_function=0, seed=42).take(1000)

# 2. 모델 초기화 함수 정의
def reset_ml_model():
    return StandardScaler() | GaussianNB()

ml_mdl = reset_ml_model()
detector = DDM()
metric = Accuracy()

# 3. 온라인 학습 루프
for i, (x, y) in enumerate(dataset):
    y_pred = ml_mdl.predict_one(x)
    is_correct = int(y_pred == y) if y_pred is not None else 1

    # 드리프트 탐지
    detector.update(is_correct)

    if detector.drift_detected:
        print(f"[{i}] Drift Detected! 모델 리셋")
        ml_mdl = reset_ml_model()
        detector = DDM()
        ml_mdl.learn_one(x, y)
        continue

    # 예측 정확도 기록 및 모델 업데이트
    metric.update(y, y_pred)
    ml_mdl.learn_one(x, y)

print("최종 정확도:", metric)

[712] Drift Detected! 모델 리셋
[781] Drift Detected! 모델 리셋
최종 정확도: Accuracy: 82.87%


AttributeError: 'DDM' object has no attribute 'change_detected'